# Transcribe Recordings

paid service [Goodtape](http://goodtape.io) will be used for the transcriptions. And [webhooks](https://webhook.site/) for retrieval of them asynchronously

## Imports

In [1]:
%reload_ext autoreload
%autoreload 2

In [3]:
import requests
import pandas as pd
import os
import docx
import os
import re
from glob import glob
import json

# local imports
from lib_henryk.logger import *
from lib_henryk.config import *
from lib_henryk.utils import *

from lib_henryk.recordings import transcriptions
from lib_henryk.recordings import recordings

## Params
You need to retrieve webhooks token ID from the [webhooks](https://webhook.site/) website. You can use this website to verify which transcriptions were made already. Notebook functions will make sure old requests that were processed are getting removed. Webhooks has a free quota of approx 100 requests. You can go around it by opening the webhooks site in the incognito window 

![webhooks-token-id](../../references/images/webhooks-token-id.png)

In [6]:
# load api keys
from dotenv import load_dotenv
_ = load_dotenv()

# retrieve new token by opening 'https://webhook.site/' in incognito mode
# and picking up the token from the URL
WEBHOOKS_TOKEN_ID = 'f4185167-7263-41d5-b39d-bd5923556c56'
GOODTAPE_API_KEY = os.getenv("GOODTAPE_API_KEY")

## Load Dataset

In [93]:
# get fresh stats directly from the recordings
df_transcriptions_log, df_transcriptions = transcriptions.init(transcriptions_parquet_path=FILE_TRANSCRIPTIONS_PARQUET)

In [94]:
df_transcriptions

,name,transcription
0,Henryk 2022-05-26 Bajka o nowym samochodziku,"Cześć Henryczku, cześć kochanie. Twój tatuś mó..."
1,Henryk 2022-05-27 Bajka o Eryku r1,"Cześć Henryczku. Mówi twój tatuś, kochanie. Ta..."
2,Henryk 2022-05-28 Bajka o swince peppie i domu...,"Cześć Henryczku, cześć kochanie, twój tatuś je..."
3,Henryk 2022-05-30 Bajka o parku dinozaurow,Cześć Henryczku. Cześć kochanie. To twój tatuś...
4,Henryk 2022-05-31 Bajka o drużynie koszykowki,"Cześć kochanie, cześć Henryczku, kochany najdr..."
...,...,...
523,Henryk 2024-06-10 Tatuś opowiada o sztucznej ...,"Cześć Henryczku, cześć kochanie, cześć mój syn..."
524,Henryk 2024-06-12 Tatuś opowiada o dniu rodzi...,Cześć mój syneczku kochany Cześć Henryczko Cze...
525,Henryk 2024-06-14 Tatuś opowiada o kotku przy...,"Cześć, Henryczku. Cześć, kochanie. Cześć, mój ..."
526,Rodzice Henryk 2024-06-15 Babcia i Dziadek opo...,"Dobrze. Halo. Dzień... Halo, obniż. Mówię do C..."


## Run Transcriptions

In [14]:
# perform transcriptions
df_transcriptions_log = transcriptions.submit_transcriptions_goodtape( 
    df_transcriptions_log=df_transcriptions_log,
    df_transcriptions=df_transcriptions,
    webhooks_token_id=WEBHOOKS_TOKEN_ID,
    goodtape_api_key=GOODTAPE_API_KEY,
    path_recordings=DIR_RECORDINGS,
    verbose=False
)

processing [87]: Henryk 2024-05-20 Tatuś opowiada o malowanou poko...
processing [88]: Henryk 2024-05-22 Tatuś opowiada o przyjęciu a Mer...
processing [89]: Henryk 2024-05-24 Tatuś opowiada o zabawkach dla p...
processing [90]: Henryk 2024-05-25 Tatuś wyjechał z wujkiem na row...
processing [91]: Henryk 2024-05-26 Tatuś opowiada o dniu Mamy a Ba...
processing [92]: Henryk 2024-05-27 Tatuś opowiada o ciężarówce ...
processing [93]: Henryk 2024-05-29 Tatuś opowiada o cioci Patrycji...
processing [94]: Henryk 2024-05-31 Tatuś opowiada o smokach i stan...
processing [95]: Henryk 2024-06-01 Tatuś opowiada o dniu dziecka i...
processing [96]: Henryk 2024-06-02 Tatuś opowiada o podróży do S...
processing [97]: Henryk 2024-06-03 Tatuś opowiada o chmurze i konf...
processing [98]: Henryk 2024-06-05 Tatuś opowiada o Krówkowie i P...
processing [99]: Henryk 2024-06-07 Tatuś opowiada o hulajnodze i n...
processing [100]: Henryk 2024-06-09 Tatuś opowiada o tym jak powsta...
proce

## Fetch Transcriptions JSONs

In [ ]:
# need to wait until transcriptions are available
exit_cell()

In [34]:
# retrieve goodtape server responses from webhooks
df_transcriptions_log, df_transcriptions = transcriptions.retrieve_responses_goodtape_via_webhooks(
    df_transcriptions_log=df_transcriptions_log,
    df_transcriptions=df_transcriptions,
    webhooks_token_id=WEBHOOKS_TOKEN_ID, 
)

found 0 requests to fetch
0 transcriptions were retrieved
*** all submitted recordings were transcribed ***


In [26]:
# process transcriptions to doc
transcriptions.transcriptions_to_doc(
    df_transcriptions=df_transcriptions, 
    doc_template_path=FILE_TRANSCRIPTION_TEMPLATE,
    path_transcriptions_doc=DIR_TRANSCRIPTIONS_DOC,
    overwrite=True,
    verbose=False,
)

processing 529 transcriptions |██████████████████████████████████████████████████| 100.0% done.
0 new transcriptions were processed, there are 529 doc transcriptions available


## Cleanup missing requests
- transcription ids for the requests that were lost will be removed, only those that were processed will remain

In [29]:
# cleanup stale transcription ids - but work on a copy of the log
df_transcriptions_to_write = transcriptions.get_verified_transcriptions(
    df_transcriptions_log=df_transcriptions_log, df_transcriptions=df_transcriptions
)

# save transcription log
df_transcriptions_to_write.to_parquet(FILE_TRANSCRIPTIONS_LOG_PARQUET, index=False)

verifying 528 transcriptions |██████████████████████████████████████████████████| 100.0% done.
there are 528 out of 528 valid transcriptions
*** all 528 recordings were transcribed ***


In [75]:
df_transcriptions = df_transcriptions.reset_index()

In [77]:
df_transcriptions.to_parquet(FILE_TRANSCRIPTIONS_PARQUET)

In [78]:
pd.read_parquet(FILE_TRANSCRIPTIONS_PARQUET)

,name,transcription
0,Henryk 2022-08-16 Koniec bajki o powstaniu Cha...,"Cześć Henryczku, cześć kochanie. To twój tatuś..."
1,Rodzice Henryk 2024-01-01 Nowy rok,"Halo, dzień dobry. Dzień dobry Henryczku. Jest..."
2,Henryk 2024-02-16 Tatuś opowiada o robocie do...,"Cześć Henryczku, cześć kochanie, cześć mój syn..."
3,Henryk 2022-06-03 Bajka o jeżyku cyprianie,"Cześć Henryczku, cześć kochanie, twój tatuś tu..."
4,Henryk 2024-01-09 Tatuś opowiada o książkac...,"Cześć Henryczku, cześć kochanie, cześć mój syn..."
...,...,...
523,Henryk 2024-05-26 Tatuś opowiada o dniu Mamy ...,"Cześć, Henryczku. Cześć, kochanie. Cześć, mój ..."
524,Henryk 2024-05-25 Tatuś wyjechał z wujkiem na...,"Cześć, Henryczku. Cześć, kochanie. Cześć, mój ..."
525,Henryk 2024-05-24 Tatuś opowiada o zabawkach d...,"Cześć, chymeczku. Cześć, kochanie. Cześć, mój ..."
526,Henryk 2024-05-20 Tatuś opowiada o malowanou ...,Cześć Henryczko Cześć kochanie Cześć mój synec...
